In [1]:
import gym
from urdfenvs.urdf_common.urdf_env import UrdfEnv
from urdfenvs.robots.generic_urdf import GenericUrdfReacher
from urdfenvs.sensors.full_sensor import FullSensor, Sensor
from urdfenvs.sensors.obstacle_sensor import ObstacleSensor
from urdfenvs.scene_examples.goal import goal1
from urdfenvs.scene_examples.obstacles import (
    sphereObst1,
    movable_obstacle,
    dynamicSphereObst3,
)
import numpy as np

from gym.wrappers.flatten_observation import FlattenObservation
from urdfenvs.urdf_common.reward import Reward


pybullet build time: May 20 2022 19:45:31


In [2]:
n_steps=300
render=False
obstacles=True
goal=True

In [15]:
class ExampleReward(Reward):
    def calculateReward(self, sensors: list[Sensor]) -> float:
        for sensor in sensors:
            print(sensor.get_observation_space())
        print("Calculating reward...")
        return 1    

In [19]:
robots = [
    GenericUrdfReacher(urdf="pointRobot.urdf", mode="vel"),
]
env= UrdfEnv(
    dt=0.01, 
    robots=robots, 
    render=render, 
    # reward= ExampleReward(),
)
env.add_obstacle(sphereObst1)
env.add_obstacle(dynamicSphereObst3)
env.add_obstacle(movable_obstacle)
env.add_goal(goal1)

In [10]:
sensor = FullSensor(['position'], ['position', 'size'], variance=0.0)
env.add_sensor(sensor, [0])
env.set_spaces()
defaultAction = np.array([0.5, -0.2, 0.0])
pos0 = np.array([1.0, 0.1, 0.0])
vel0 = np.array([1.0, 0.0, 0.0])

In [17]:
initial_observations = []
for _ in range(10):
    ob = env.reset(pos=pos0, vel=vel0)
    env.shuffle_goals()
    env.shuffle_obstacles()
    initial_observations.append(ob)
    print(f"Initial observation : {ob}")

    history = []
    for _ in range(n_steps):
        action = defaultAction
        ob, reward, done, info = env.step(action)
        # In observations, information about obstacles is stored in ob['obstacleSensor']
        history.append(ob)

Initial observation : {'robot_0': {'joint_state': {'position': array([1. , 0.1, 0. ]), 'velocity': array([1., 0., 0.])}}}
Calculating reward...
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
lidar_sensor_linkCalculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...
Calculating reward...

In [8]:
env.observation_space

Dict(robot_0:Dict(FullSensor:Dict(goals:Dict(3:Dict(position:Box(-5.0, 5.0, (3,), float32)), 7:Dict(position:Box(-5.0, 5.0, (3,), float32))), obstacles:Dict(1:Dict(position:Box(-5.0, 5.0, (3,), float32), size:Box(0.0, 5.0, (1,), float32)), 2:Dict(position:Box(-5.0, 5.0, (3,), float32), size:Box(0.0, 5.0, (3,), float32)), 5:Dict(position:Box(-5.0, 5.0, (3,), float32), size:Box(0.0, 5.0, (1,), float32)), 6:Dict(position:Box(-5.0, 5.0, (3,), float32), size:Box(0.0, 5.0, (3,), float32)))), joint_state:Dict(position:Box(-5.0, 5.0, (3,), float64), velocity:Box(-2.175, 2.175, (3,), float64))))

In [9]:
history

[{'robot_0': {'joint_state': {'position': array([ 1.00500000e+00,  9.80000000e-02, -8.94552064e-06]),
    'velocity': array([ 5.00000000e-01, -2.00000000e-01,  5.50813688e-07])},
   'FullSensor': {'obstacles': {1: {'position': array([1.480438e-05, 1.000000e+00, 1.000000e+00], dtype=float32),
      'size': array([0.7147409], dtype=float32)},
     2: {'position': array([1.1109885 , 0.3769296 , 0.71000266], dtype=float32),
      'size': array([0.2, 0.2, 0.2], dtype=float32)},
     5: {'position': array([1.480438e-05, 1.000000e+00, 1.000000e+00], dtype=float32),
      'size': array([0.7147409], dtype=float32)},
     6: {'position': array([1.4299432 , 0.3769296 , 0.71000266], dtype=float32),
      'size': array([0.2, 0.2, 0.2], dtype=float32)}},
    'goals': {3: {'position': array([0.6189669 , 0.37784296, 0.80760795], dtype=float32)},
     7: {'position': array([0.6189669 , 0.37784296, 0.80760795], dtype=float32)}}}}},
 {'robot_0': {'joint_state': {'position': array([ 1.01000000e+00,  9.600

In [ ]:
env.close()